In [1]:
import os
import sys
import subprocess
import pickle
import numpy as np
import ase.io as aseIO
import time

def getMinIdx(filename):
    """
        Get the optimal hyperparameters
        
        ---Arguments---
        filename: file containing the error matrix
    """
    
    # Load the matrix
    x = np.load(filename)
    
    # The errors for all width and sigma
    # (uses first training point, so we
    # should optimize using just one training set size)
    xErr = x[0, :, :, 0, 0, 5]
    
    # Get minimum error
    xMin = np.argmin(xErr)
    
    # Get the 2D index from the 1D index
    nCol = x.shape[2]
    pos = (xMin/nCol, xMin%nCol)
    
    # Output width, sigma
    return (x[0, pos[0], pos[1], 0, 0, 1],
            x[0, pos[0], pos[1], 0, 0, 2])

# Initialize workflow parameters
sampleSizes = ['1000', '10000']
sampleNames = ['1k', '10k']
properties = ['Energy_per_Si', 'volume']
propertyNames = ['Energy', 'Volume']
cutoffs = ['3.5', '6.0']
kernels = ['gaussian', 'linear']
kernelNames = ['Gaussian', 'Linear']
maxTrain = ['800', '8000']
trainPts = {'10k': ['10', '30', '50', '100', '300', '500', '1000', '3000', '5000', '8000'],
           '1k': ['10', '30', '50', '100', '300', '500', '800']}
pcaPts = {'Energy': ['1', '2', '4', '10', '20', '100', '300'],
         'Volume': ['1', '2', '4', '10', '20', '50', '100']}

# Load the hyperparamters, if they exist
try:
    parameterFile = open('hyperparameters.pkl', 'rb')
    print 'Found hyperparameter file'
    hypers = pickle.load(parameterFile)
    parameterFile.close()
except IOError:
    hypers = {}
    print 'Creating new hyperparameters'

Found hyperparameter file


In [2]:
print hypers

{'10k-KingB-E-L': (nan, 1e-05), '10k-PCA3.5-E-L': (nan, 0.01), '10k-KingB-E-G': (3.0, 1.0), '10k-PCA3.5-E-G': (0.3, 0.03), '10k-PCA6.0-V-G': (1.0, 0.01), '10k-Ang-V-G': (10.0, 0.1), '10k-KingD-V-L': (nan, 0.03), '10k-PCA6.0-V-L': (nan, 0.3), '1k-PCA3.5-V-L': (nan, 1.0), '10k-Ang-V-L': (nan, 0.1), '10k-SOAP3.5-E-G': (0.3, 0.03), '10k-KPCA3.5-V-G': (1.0, 0.03), '10k-SOAP3.5-E-L': (nan, 0.001), '10k-KPCA3.5-V-L': (nan, 0.3), '1k-KPCA6.0-V-G': (1.0, 0.03), '1k-KPCA6.0-V-L': (nan, 1.0), '1k-ShortD-E-L': (nan, 0.3), '10k-SOAP6.0-V-G': (1.0, 0.01), '10k-Dist-V-L': (nan, 0.03), '1k-ShortD-E-G': (100.0, 0.01), '10k-Ang-E-L': (nan, 0.001), '10k-Ang-E-G': (30.0, 0.1), '1k-SOAP6.0-E-G': (3.0, 0.003), '10k-KPCA6.0-E-L': (nan, 0.1), '10k-KPCA3.5-E-L': (nan, 0.1), '10k-KPCA6.0-E-G': (1.0, 0.03), '10k-KPCA3.5-E-G': (1.0, 0.1), '1k-SOAP3.5-E-L': (nan, 0.01), '10k-Dist-V-G': (0.1, 0.1), '1k-SOAP3.5-E-G': (0.3, 0.03), '10k-KingB-V-G': (3.0, 0.1), '1k-PCA6.0-V-G': (1.0, 0.03), '10k-SOAP6.0-E-L': (nan, 0.0

# Sort ring files (done)

In [2]:
files = ['King_Binary', 'King_Distribution',
        'Short_Binary', 'Short_Distribution']

for ss in sampleSizes:
    for f in files:
        print 'Current construction: %s %s' % (ss, f)
        inFile = '../Raw_Data/DEEM_%s_Rings_%s.xyz' % (ss, f)
        outFile = '../Raw_Data/DEEM_%s_Rings_%s_Sorted.xyz' % (ss, f)
        subprocess.call(['python', 'sort_rings.py',
                        '-input', inFile,
                        '-output', outFile])

# Make k-Folds (done)

In [3]:
for ss, sn in zip(sampleSizes, sampleNames):
    print 'Current construction: %s' % sn
    workDir = '../Processed_Data/DEEM_%s' % sn
    
    # Create directories
    subprocess.call(['mkdir', '-p', workDir])
    
    subprocess.call(['python', 'kFolds.py',
                    '-nt', ss,
                    '-f', '1.0',
                    '-k', '5',
                    '-output', workDir])

# Compute SOAP vectors

## Compute all SOAP vectors (1k only) (done)

In [9]:
for c in cutoffs:
    print 'Current construction: %s' % c
    structureFile = '../Raw_Data/DEEM_1000.xyz'
    workDir = '../Processed_Data/DEEM_1k/PCA/%s' % c
    
    # Create directories
    subprocess.call(['mkdir', '-p', workDir])
        
    # Compute full SOAP vectors
    subprocess.call(['python', 'SOAP.py',
                    '-structure', structureFile,
                    '-n', '12',
                    '-l', '9',
                    '-c', c,
                    '-cw', '0.3',
                    '-g', '0.3',
                    '-Z', '14',
                    '-z', '14', '8',
                    '-output', workDir])

    # Select FPS components
    subprocess.call(['python', 'FPS.py',
                    '-soap', '%s/SOAPFiles.dat' % workDir,
                    '-fps', '500',
                    '-c',
                    '-output', workDir])
    
    os.rename('%s/FPS.idxs' % workDir, 
              '%s/FPS-c.idxs' % workDir)
    
    # Recompute SOAPs, retain only the FPS components
    subprocess.call(['python', 'SOAP.py',
                    '-structure', structureFile,
                    '-n', '12',
                    '-l', '9',
                    '-c', c,
                    '-cw', '0.3',
                    '-g', '0.3',
                    '-Z', '14',
                    '-z', '14', '8',
                    '-idxs', '%s/FPS-c.idxs' % workDir,
                    '-output', workDir])

    # Select representative environments
    subprocess.call(['python', 'FPS.py',
                    '-soap', '%s/SOAPFiles.dat' % workDir,
                    '-fps', '2000',
                    '-output', workDir])
    
    os.rename('%s/FPS.idxs' % workDir,
              '%s/FPS-rSOAP.idxs' % workDir)

## Compute SOAP vectors (10k only) (done)

In [11]:
for c in cutoffs:
    print 'Current construction: %s' % c
    structureFile = '../Raw_Data/DEEM_10000.xyz'
    workDir = '../Processed_Data/DEEM_10k/PCA/%s' % c
    
    # Create directories
    subprocess.call(['mkdir', '-p', workDir])
    
    # Select random structures
    subprocess.call(['python', 'randomStructureSelect.py',
                    '-structure', structureFile,
                    '-nt', '10000',
                    '-nr', '2000',
                    '-output', workDir])

    # Compute SOAP vectors for random structures
    subprocess.call(['python', 'SOAP.py',
                    '-structure', '%s/randomSelection.xyz' % workDir,
                    '-n', '12',
                    '-l', '9',
                    '-c', c,
                    '-cw', '0.3',
                    '-g', '0.3',
                    '-Z', '14',
                    '-z', '14', '8',
                    '-output', workDir])

    # Select FPS components
    subprocess.call(['python', 'FPS.py',
                    '-soap', '%s/SOAPFiles.dat' % workDir,
                    '-fps', '500',
                    '-c',
                    '-output', workDir])
    
    os.rename('%s/FPS.idxs' % workDir,
              '%s/FPS-c.idxs' % workDir)

    # Re-compute SOAPs, retain only FPS components
    subprocess.call(['python', 'SOAP.py',
                    '-structure', structureFile,
                    '-n', '12',
                    '-l', '9',
                    '-c', c,
                    '-cw', '0.3',
                     '-g', '0.3',
                    '-Z', '14',
                    '-z', '14', '8',
                    '-idxs', '%s/FPS-c.idxs' % workDir,
                    '-batchsize', '1000',
                    '-output', workDir])

    # Select representative environments
    subprocess.call(['python', 'FPS.py',
                    '-soap', '%s/SOAPFiles.dat' % workDir,
                    '-fps', '2000',
                    '-output', workDir])
    
    os.rename('%s/FPS.idxs' % workDir,
              '%s/FPS-rSOAP.idxs' % workDir)

## Hyperparameter optimization (done)

In [10]:
for ss, sn, mt in zip(sampleSizes, sampleNames, maxTrain):
    for c in cutoffs:
        for p, pn in zip(properties, propertyNames):
            for k, kn in zip(kernels, kernelNames):
                print 'Current optimization: %s %s %s %s' % (sn, c, pn, kn)
                workDir = '../Processed_Data/DEEM_%s/%s/%s/ParameterSearch/%s' % (sn, pn, c, kn)
                dataDir = '../Processed_Data/DEEM_%s/PCA/%s' % (sn, c)
                foldDir = '../Processed_Data/DEEM_%s' % sn
                structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                dictKey = '%s-SOAP%s-%s-%s' % (sn, c, pn[0], kn[0])
                
                # Create directories
                subprocess.call(['mkdir', '-p', workDir])

                subprocess.call(['python', 'learningCurves.py',
                                '-structure', structureFile,
                                '-soap', '%s/SOAPFiles.dat' % dataDir,
                                '-idxs', '%s/FPS-rSOAP.idxs' % dataDir,
                                '-p', p,
                                '-Z', '14',
                                '-k', '5',
                                '-kernel', k,
                                '-width', '0.001', '0.003', '0.01', '0.03', 
                                          '0.1', '0.3', '1.0', '3.0', '10.0',
                                '-sigma', '0.001', '0.003', '0.01', '0.03', 
                                          '0.1', '0.3', '1.0', '3.0', '10.0',
                                '-zeta', '1',
                                '-ntrain', mt,
                                '-train', '%s/kTrain.idxs' % foldDir,
                                '-validate', '%s/kValidate.idxs' % foldDir,
                                '-shuffle',
                                '-output', workDir])
                optParams = getMinIdx('%s/maeAvgTest.npy' % workDir)
                hypers[dictKey] = optParams

parameterFile = open('hyperparameters.pkl', 'wb')
pickle.dump(hypers, parameterFile)
parameterFile.close()

Current optimization: 1k 3.5 Energy Gaussian
Current optimization: 1k 3.5 Energy Linear
Current optimization: 1k 3.5 Volume Gaussian
Current optimization: 1k 3.5 Volume Linear
Current optimization: 1k 6.0 Energy Gaussian
Current optimization: 1k 6.0 Energy Linear
Current optimization: 1k 6.0 Volume Gaussian
Current optimization: 1k 6.0 Volume Linear
Current optimization: 10k 3.5 Energy Gaussian
Current optimization: 10k 3.5 Energy Linear
Current optimization: 10k 3.5 Volume Gaussian
Current optimization: 10k 3.5 Volume Linear
Current optimization: 10k 6.0 Energy Gaussian
Current optimization: 10k 6.0 Energy Linear
Current optimization: 10k 6.0 Volume Gaussian
Current optimization: 10k 6.0 Volume Linear


# Build PCA

## Build linear PCA (done)

In [3]:
for ss, sn in zip(sampleSizes, sampleNames):
    for c in cutoffs:
        print 'Current construction: %s %s' % (sn, c)
        workDir = '../Processed_Data/DEEM_%s/PCA/%s' % (sn, c)
        
        # Create directories
        subprocess.call(['mkdir', '-p', workDir])
        
        subprocess.call(['python', 'SOAP-PCA.py',
                        '-soap', '%s/SOAPFiles.dat' % workDir,
                        '-dopca',
                        '-pca', '500',
                        '-output', workDir])

        subprocess.call(['python', 'SOAP-PCA.py',
                        '-soap', '%s/SOAPFiles.dat' % workDir,
                        '-dotransform',
                        '-w', '%s/eigenvectors.dat' % workDir,
                        '-mean', '%s/mean.dat' % workDir,
                        '-output', workDir])

        subprocess.call(['python', 'FPS.py',
                        '-soap', '%s/PCAFiles.dat' % workDir,
                        '-fps', '2000',
                        '-output', workDir])
        
        os.rename('%s/FPS.idxs' % workDir,
                  '%s/FPS-rPCA.idxs' % workDir)

Current construction: 1k 3.5
Current construction: 1k 6.0
Current construction: 10k 3.5
Current construction: 10k 6.0


## Build KPCA (done)

In [3]:
for ss, sn in zip(sampleSizes, sampleNames):
    for c in cutoffs:
        print 'Current construction: %s %s' % (sn, c)
        dictKey = '%s-SOAP%s-%s-%s' % (sn, c, 'V', 'G') # Use volume as default choice to build the KPCA
        workDir = '../Processed_Data/DEEM_%s/PCA/%s' % (sn, c)
        
        # Create directories
        subprocess.call(['mkdir', '-p', workDir])
        
        subprocess.call(['python', 'centerData.py',
                        '-soap', '%s/SOAPFiles.dat' % workDir])

        subprocess.call(['python', 'SOAP-KPCA.py',
                        '-soap', '%s/SOAPFiles-centered.dat' % workDir,
                        '-pca', '500',
                        '-kernel', 'gaussian', # Don't need linear b/c we have the PCA, which is faster
                        '-width', str(hypers[dictKey][0]),
                        '-lowmem',
                        '-idxs', '%s/FPS-rSOAP.idxs' % workDir,
                        '-output', workDir])

        subprocess.call(['python', 'FPS.py',
                        '-soap', '%s/KPCAFiles.dat' % workDir,
                        '-fps', '2000',
                        '-output', workDir])
        
        os.rename('%s/FPS.idxs' % workDir,
                  '%s/FPS-rKPCA.idxs' % workDir)

Current construction: 1k 3.5
Current construction: 1k 6.0
Current construction: 10k 3.5
Current construction: 10k 6.0


# Build classical descriptors

## Distances (done)

In [4]:
for ss, sn in zip(sampleSizes, sampleNames):
    print 'Current construction: %s' % sn
    workDir = '../Processed_Data/DEEM_%s/Distances' % sn
    structureFile = '../Raw_Data/DEEM_%s_Distances.xyz' % ss
    
    # Create directories
    subprocess.call(['mkdir', '-p', workDir])
    
    subprocess.call(['python', 'buildClassicalDescriptors.py',
                    '-input', structureFile,
                    '-output', '%s/distances.dat' % workDir,
                    '-p', 'distances'])
    
    f = open('%s/distanceFiles.dat' % workDir, 'w')
    f.write(os.path.abspath('%s/distances.dat' % workDir))
    f.close()
    
    subprocess.call(['python', 'FPS.py',
                    '-soap', '%s/distanceFiles.dat' % workDir,
                    '-fps', '2000',
                    '-output', workDir])
        
    os.rename('%s/FPS.idxs' % workDir,
              '%s/FPS-rDistances.idxs' % workDir)

Current construction: 1k
Current construction: 10k


## Angles (done)

In [6]:
for ss, sn in zip(sampleSizes, sampleNames):
    print 'Current construction: %s' % sn
    workDir = '../Processed_Data/DEEM_%s/Angles' % sn
    structureFile = '../Raw_Data/DEEM_%s_Angles.xyz' % ss
    
    # Create directories
    subprocess.call(['mkdir', '-p', workDir])
    
    subprocess.call(['python', 'buildClassicalDescriptors.py',
                    '-input', structureFile,
                    '-output', '%s/angles.dat' % workDir,
                    '-p', 'angles'])
    
    f = open('%s/angleFiles.dat' % workDir, 'w')
    f.write(os.path.abspath('%s/angles.dat' % workDir))
    f.close()
    
    subprocess.call(['python', 'FPS.py',
                    '-soap', '%s/angleFiles.dat' % workDir,
                    '-fps', '2000',
                    '-output', workDir])
        
    os.rename('%s/FPS.idxs' % workDir,
              '%s/FPS-rAngles.idxs' % workDir)

Current construction: 1k
Current construction: 10k


## Rings (done)

In [9]:
for ss, sn in zip(sampleSizes, sampleNames):
    for ks in ['King', 'Short']:
        for bd in ['Binary', 'Distribution']:
            print 'Current construction: %s %s %s' % (sn, ks, bd)
            workDir = '../Processed_Data/DEEM_%s/Rings/%s/%s' % (sn, ks, bd)
            structureFile = '../Raw_Data/DEEM_%s_Rings_%s_%s_Sorted.xyz' % (ss, ks, bd)
            
            # Create directories
            subprocess.call(['mkdir', '-p', workDir])
            
            subprocess.call(['python', 'buildClassicalDescriptors.py',
                            '-input', structureFile,
                            '-output', '%s/rings.dat' % workDir,
                            '-p', 'rings'])
            
            f = open('%s/ringFiles.dat' % workDir, 'w')
            f.write(os.path.abspath('%s/rings.dat' % workDir))
            f.close()
            
            subprocess.call(['python', 'FPS.py',
                    '-soap', '%s/ringFiles.dat' % workDir,
                    '-fps', '2000',
                    '-output', workDir])
        
            os.rename('%s/FPS.idxs' % workDir,
                      '%s/FPS-rRings.idxs' % workDir)

Current construction: 1k King Binary
Current construction: 1k King Distribution
Current construction: 1k Short Binary
Current construction: 1k Short Distribution
Current construction: 10k King Binary
Current construction: 10k King Distribution
Current construction: 10k Short Binary
Current construction: 10k Short Distribution


In [10]:
# Clean up the FPS files; get rid of repeated ('0') indices
# (as a result of there being fewer unique environments
# than desired representative environments)
for ss, sn in zip(sampleSizes, sampleNames):
    for ks in ['King', 'Short']:
        for bd in ['Binary', 'Distribution']:
            workDir = '../Processed_Data/DEEM_%s/Rings/%s/%s' % (sn, ks, bd)
            idxs = np.loadtxt('%s/FPS-rRings.idxs' % workDir)
            newIdxs, locations = np.unique(idxs, return_index=True)
            if len(newIdxs) < len(idxs):
                locations.sort()
                os.rename('%s/FPS-rRings.idxs' % workDir,
                         '%s/FPS-rRings.idxs.bk' % workDir)
                np.savetxt('%s/FPS-rRings.idxs' % workDir, idxs[locations], fmt='%d')

# Hyperparameter Optimization

## PCA (done)

In [16]:
for ss, sn, mt in zip(sampleSizes, sampleNames, maxTrain):
    for c in cutoffs:
        for p, pn in zip(properties, propertyNames):
            for k, kn in zip(kernels, kernelNames):
                print 'Current optimization: %s %s %s %s' % (sn, c, pn, kn)
                workDir = '../Processed_Data/DEEM_%s/%s/%s/PCALearn/ParameterSearch/%s' % (sn, pn, c, kn)
                structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                dataFile = '../Processed_Data/DEEM_%s/PCA/%s/PCAFiles.dat' % (sn, c)
                idxsFile = '../Processed_Data/DEEM_%s/PCA/%s/FPS-rPCA.idxs' % (sn, c)
                trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
                validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
                dictKey = '%s-PCA%s-%s-%s' % (sn, c, pn[0], kn[0])
                
                # Create directories
                subprocess.call(['mkdir', '-p', workDir])

                subprocess.call(['python', 'learningCurves.py',
                                '-structure', structureFile,
                                '-soap', dataFile,
                                '-idxs', idxsFile,
                                '-p', p,
                                '-Z', '14',
                                '-k', '5',
                                '-kernel', k,
                                '-width', '0.001', '0.003', '0.01', '0.03', 
                                          '0.1', '0.3', '1.0', '3.0', '10.0',
                                '-sigma', '0.001', '0.003', '0.01', '0.03', 
                                          '0.1', '0.3', '1.0', '3.0', '10.0', 
                                '-zeta', '1',
                                '-ntrain', mt,
                                '-train', trainFile,
                                '-validate', validateFile,
                                '-shuffle',
                                '-output', workDir])
                optParams = getMinIdx('%s/maeAvgTest.npy' % workDir)
                hypers[dictKey] = optParams

parameterFile = open('hyperparameters.pkl', 'wb')
pickle.dump(hypers, parameterFile)
parameterFile.close()

Current optimization: 1k 3.5 Energy Linear
Current optimization: 1k 3.5 Volume Linear
Current optimization: 1k 6.0 Energy Linear
Current optimization: 1k 6.0 Volume Linear
Current optimization: 10k 3.5 Energy Linear
Current optimization: 10k 3.5 Volume Linear
Current optimization: 10k 6.0 Energy Linear
Current optimization: 10k 6.0 Volume Linear


## KPCA (done)

In [6]:
for ss, sn, mt in zip(sampleSizes, sampleNames, maxTrain):
    for c in cutoffs:
        for p, pn in zip(properties, propertyNames):
            for k, kn in zip(kernels, kernelNames):
                print 'Current optimization: %s %s %s %s' % (sn, c, pn, kn)
                workDir = '../Processed_Data/DEEM_%s/%s/%s/KPCALearn/ParameterSearch/%s' % (sn, pn, c, kn)
                structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                dataFile = '../Processed_Data/DEEM_%s/PCA/%s/KPCAFiles.dat' % (sn, c)
                idxsFile = '../Processed_Data/DEEM_%s/PCA/%s/FPS-rKPCA.idxs' % (sn, c)
                trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
                validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
                dictKey = '%s-KPCA%s-%s-%s' % (sn, c, pn[0], kn[0])
                
                # Create directories
                subprocess.call(['mkdir', '-p', workDir])

                subprocess.call(['python', 'learningCurves.py',
                                '-structure', structureFile,
                                '-soap', dataFile,
                                '-idxs', idxsFile,
                                '-p', p,
                                '-Z', '14',
                                '-k', '5',
                                '-kernel', k,
                                '-width', '0.001', '0.003', '0.01', '0.03', 
                                          '0.1', '0.3', '1.0', '3.0', '10.0',
                                '-sigma', '0.001', '0.003', '0.01', '0.03', 
                                          '0.1', '0.3', '1.0', '3.0', '10.0',
                                '-zeta', '1', 
                                '-ntrain', mt,
                                '-train', trainFile,
                                '-validate', validateFile,
                                '-shuffle',
                                '-output', workDir])
                optParams = getMinIdx('%s/maeAvgTest.npy' % workDir)
                hypers[dictKey] = optParams                

                parameterFile = open('hyperparameters.pkl', 'wb')
pickle.dump(hypers, parameterFile)
parameterFile.close()

Current optimization: 1k 3.5 Energy Gaussian
Current optimization: 1k 3.5 Energy Linear
Current optimization: 1k 3.5 Volume Gaussian
Current optimization: 1k 3.5 Volume Linear
Current optimization: 1k 6.0 Energy Gaussian
Current optimization: 1k 6.0 Energy Linear
Current optimization: 1k 6.0 Volume Gaussian
Current optimization: 1k 6.0 Volume Linear
Current optimization: 10k 3.5 Energy Gaussian
Current optimization: 10k 3.5 Energy Linear
Current optimization: 10k 3.5 Volume Gaussian
Current optimization: 10k 3.5 Volume Linear
Current optimization: 10k 6.0 Energy Gaussian
Current optimization: 10k 6.0 Energy Linear
Current optimization: 10k 6.0 Volume Gaussian
Current optimization: 10k 6.0 Volume Linear


## Distances (done)

In [4]:
for ss, sn, mt in zip(sampleSizes, sampleNames, maxTrain):
    for p, pn in zip(properties, propertyNames):
        for k, kn in zip(kernels, kernelNames):
            print 'Current optimization: %s %s %s' % (sn, pn, kn)
            workDir = '../Processed_Data/DEEM_%s/%s/Distances/ParameterSearch/%s' % (sn, pn, kn)
            structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
            soapFile = '../Processed_Data/DEEM_%s/Distances/distanceFiles.dat' % sn
            idxsFile = '../Processed_Data/DEEM_%s/Distances/FPS-rDistances.idxs' % sn
            trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
            validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
            dictKey = '%s-Dist-%s-%s' % (sn, pn[0], kn[0])
            
            # Create directories
            subprocess.call(['mkdir', '-p', workDir])

            subprocess.call(['python', 'learningCurves.py',
                            '-structure', structureFile,
                            '-soap', soapFile,
                            '-idxs', idxsFile,
                            '-p', p,
                            '-Z', '14',
                            '-k', '5',
                            '-kernel', k,
                            '-width', '0.001', '0.003', '0.01', '0.03', 
                                      '0.1', '0.3', '1.0',
                            '-sigma', '0.001', '0.003', '0.01', '0.03', 
                                      '0.1', '0.3', '1.0',
                            '-zeta', '1',
                            '-ntrain', mt,
                            '-train', trainFile,
                            '-validate', validateFile,
                            '-shuffle',
                            '-output', workDir])
            optParams = getMinIdx('%s/maeAvgTest.npy' % workDir)
            hypers[dictKey] = optParams            

parameterFile = open('hyperparameters.pkl', 'wb')
pickle.dump(hypers, parameterFile)
parameterFile.close()

Current optimization: 1k Energy Gaussian
Current optimization: 1k Energy Linear
Current optimization: 1k Volume Gaussian
Current optimization: 1k Volume Linear
Current optimization: 10k Energy Gaussian
Current optimization: 10k Energy Linear
Current optimization: 10k Volume Gaussian
Current optimization: 10k Volume Linear


## Angles (done)

In [6]:
for ss, sn, mt in zip(sampleSizes, sampleNames, maxTrain):
    for p, pn in zip(properties, propertyNames):
        for k, kn in zip(kernels, kernelNames):
            print 'Current optimization: %s %s %s' % (sn, pn, kn)
            workDir = '../Processed_Data/DEEM_%s/%s/Angles/ParameterSearch/%s' % (sn, pn, kn)
            structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
            dataFile = '../Processed_Data/DEEM_%s/Angles/angleFiles.dat' % sn
            idxsFile = '../Processed_Data/DEEM_%s/Angles/FPS-rAngles.idxs' % sn
            trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
            validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
            dictKey = '%s-Ang-%s-%s' % (sn, pn[0], kn[0])
            
            # Create directories
            subprocess.call(['mkdir', '-p', workDir])

            subprocess.call(['python', 'learningCurves.py',
                            '-structure', structureFile,
                            '-soap', dataFile,
                            '-idxs', idxsFile,
                            '-p', p,
                            '-Z', '14',
                            '-k', '5',
                            '-kernel', k,
                            '-width', '1.0', '3.0', '10.0', '30.0', 
                                      '100.0', '300.0', '1000.0',
                            '-sigma', '0.001', '0.003', '0.01', '0.03', 
                                      '0.1', '0.3', '1.0',
                            '-zeta', '1',
                            '-ntrain', mt,
                            '-train', trainFile,
                            '-validate', validateFile,
                            '-shuffle',
                            '-output', workDir])
            optParams = getMinIdx('%s/maeAvgTest.npy' % workDir)
            hypers[dictKey] = optParams            

parameterFile = open('hyperparameters.pkl', 'wb')
pickle.dump(hypers, parameterFile)
parameterFile.close()

Current optimization: 1k Energy Gaussian
Current optimization: 1k Energy Linear
Current optimization: 1k Volume Gaussian
Current optimization: 1k Volume Linear
Current optimization: 10k Energy Gaussian
Current optimization: 10k Energy Linear
Current optimization: 10k Volume Gaussian
Current optimization: 10k Volume Linear


## Rings (done)

In [9]:
for ss, sn, mt in zip(sampleSizes, sampleNames, maxTrain):
    for p, pn in zip(properties, propertyNames):
        for ks in ['King', 'Short']:
            for bd in ['Binary', 'Distribution']:
                for k, kn in zip(kernels, kernelNames):
                    print 'Current optimization: %s %s %s %s %s' % (sn, pn, ks, bd, kn)
                    workDir = '../Processed_Data/DEEM_%s/%s/Rings/%s/%s/ParameterSearch/%s' % (sn, pn, ks, bd, kn)
                    structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                    dataFile = '../Processed_Data/DEEM_%s/Rings/%s/%s/ringFiles.dat' % (sn, ks, bd)
                    idxsFile = '../Processed_Data/DEEM_%s/Rings/%s/%s/FPS-rRings.idxs' % (sn, ks, bd)
                    trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
                    validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
                    dictKey = '%s-%s%s-%s-%s' % (sn, ks, bd[0], pn[0], kn[0])
                    
                    # Create directories
                    subprocess.call(['mkdir', '-p', workDir])

                    subprocess.call(['python', 'learningCurves.py',
                                    '-structure', structureFile,
                                    '-soap', dataFile,
                                    '-idxs', idxsFile,
                                    '-p', p,
                                    '-Z', '14',
                                    '-k', '5', 
                                    '-kernel', k,
                                    '-width', '1.0', '3.0', '10.0', '100.0', 
                                              '300.0', '1000.0',
                                    '-sigma', '1E-5', '3E-5', '1E-4', '3E-4', 
                                              '0.001', '0.003', '0.01', '0.03', 
                                              '0.1', '0.3', '1.0',
                                    '-zeta', '1',
                                    '-ntrain', mt,
                                    '-train', trainFile,
                                    '-validate', validateFile,
                                    '-shuffle',
                                    '-output', workDir])
                    optParams = getMinIdx('%s/maeAvgTest.npy' % workDir)
                    hypers[dictKey] = optParams                    

parameterFile = open('hyperparameters.pkl', 'wb')
pickle.dump(hypers, parameterFile)
parameterFile.close()

Current optimization: 1k Energy King Binary Gaussian
Current optimization: 1k Energy King Binary Linear
Current optimization: 1k Energy King Distribution Gaussian
Current optimization: 1k Energy King Distribution Linear
Current optimization: 1k Energy Short Binary Gaussian
Current optimization: 1k Energy Short Binary Linear
Current optimization: 1k Energy Short Distribution Gaussian
Current optimization: 1k Energy Short Distribution Linear
Current optimization: 1k Volume King Binary Gaussian
Current optimization: 1k Volume King Binary Linear
Current optimization: 1k Volume King Distribution Gaussian
Current optimization: 1k Volume King Distribution Linear
Current optimization: 1k Volume Short Binary Gaussian
Current optimization: 1k Volume Short Binary Linear
Current optimization: 1k Volume Short Distribution Gaussian
Current optimization: 1k Volume Short Distribution Linear
Current optimization: 10k Energy King Binary Gaussian
Current optimization: 10k Energy King Binary Linear
Curren

# Learning Curves

## SOAP (done)

In [4]:
for ss, sn in zip(sampleSizes, sampleNames):
    for c in cutoffs:
        for p, pn in zip(properties, propertyNames):
            for k, kn in zip(kernels, kernelNames):
                print 'Current model: %s %s %s %s' % (sn, c, pn, kn)
                workDir = '../Processed_Data/DEEM_%s/%s/%s/%s' % (sn, pn, c, kn)
                structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                dataFile = '../Processed_Data/DEEM_%s/PCA/%s/SOAPFiles.dat' % (sn, c)
                idxsFile = '../Processed_Data/DEEM_%s/PCA/%s/FPS-rSOAP.idxs' % (sn, c)
                trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
                validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
                dictKey = '%s-SOAP%s-%s-%s' % (sn, c, pn[0], kn[0])
                
                # Create directories
                subprocess.call(['mkdir', '-p', workDir])
                
                args = ['python', 'learningCurves.py',
                        '-structure', structureFile,
                        '-soap', dataFile,
                        '-idxs', idxsFile,
                        '-p', p,
                        '-Z', '14',
                        '-k', '5',
                        '-kernel', k,
                        '-width', str(hypers[dictKey][0]),
                        '-sigma', str(hypers[dictKey][1]),
                        '-zeta', '1',
                        '-ntrain'] + trainPts[sn] + \
                        ['-train', trainFile,
                        '-validate', validateFile,
                        '-shuffle',
                        '-output', workDir]
                subprocess.call(args)

Current model: 1k 3.5 Energy Gaussian
Current model: 1k 3.5 Energy Linear
Current model: 1k 3.5 Volume Gaussian
Current model: 1k 3.5 Volume Linear
Current model: 1k 6.0 Energy Gaussian
Current model: 1k 6.0 Energy Linear
Current model: 1k 6.0 Volume Gaussian
Current model: 1k 6.0 Volume Linear
Current model: 10k 3.5 Energy Gaussian
Current model: 10k 3.5 Energy Linear
Current model: 10k 3.5 Volume Gaussian
Current model: 10k 3.5 Volume Linear
Current model: 10k 6.0 Energy Gaussian
Current model: 10k 6.0 Energy Linear
Current model: 10k 6.0 Volume Gaussian
Current model: 10k 6.0 Volume Linear


## PCA (done)

In [7]:
for ss, sn in zip(sampleSizes, sampleNames):
    for c in cutoffs:
        for p, pn in zip(properties, propertyNames):
            for k, kn in zip(kernels, kernelNames):
                print 'Current model: %s %s %s %s' % (sn, c, pn, kn)
                workDir = '../Processed_Data/DEEM_%s/%s/%s/PCALearn/%s' % (sn, pn, c, kn)
                structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                dataFile = '../Processed_Data/DEEM_%s/PCA/%s/PCAFiles.dat' % (sn, c)
                idxsFile = '../Processed_Data/DEEM_%s/PCA/%s/FPS-rPCA.idxs' % (sn, c)
                trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
                validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
                dictKey = '%s-PCA%s-%s-%s' % (sn, c, pn[0], kn[0])
                
                # Create directories
                subprocess.call(['mkdir', '-p', workDir])

                args = ['python', 'learningCurves.py',
                        '-structure', structureFile,
                        '-soap', dataFile,
                        '-idxs', idxsFile,
                        '-p', p,
                        '-Z', '14',
                        '-k', '5',
                        '-kernel', k,
                        '-width', str(hypers[dictKey][0]),
                        '-sigma', str(hypers[dictKey][1]),
                        '-zeta', '1',
                        '-ntrain'] + trainPts[sn] + \
                        ['-pcalearn'] + pcaPts[pn] + \
                        ['-train', trainFile,
                        '-validate', validateFile,
                        '-shuffle',
                        '-output', workDir]
                subprocess.call(args)

Current model: 1k 3.5 Energy Gaussian
Current model: 1k 3.5 Energy Linear
Current model: 1k 3.5 Volume Gaussian
Current model: 1k 3.5 Volume Linear
Current model: 1k 6.0 Energy Gaussian
Current model: 1k 6.0 Energy Linear
Current model: 1k 6.0 Volume Gaussian
Current model: 1k 6.0 Volume Linear
Current model: 10k 3.5 Energy Gaussian
Current model: 10k 3.5 Energy Linear
Current model: 10k 3.5 Volume Gaussian
Current model: 10k 3.5 Volume Linear
Current model: 10k 6.0 Energy Gaussian
Current model: 10k 6.0 Energy Linear
Current model: 10k 6.0 Volume Gaussian
Current model: 10k 6.0 Volume Linear


## KPCA (done)

In [3]:
for ss, sn in zip(sampleSizes, sampleNames):
    for c in cutoffs:
        for p, pn in zip(properties, propertyNames):
            for k, kn in zip(kernels, kernelNames):
                print 'Current model: %s %s %s %s' % (sn, c, pn, kn)
                workDir = '../Processed_Data/DEEM_%s/%s/%s/KPCALearn/%s' % (sn, pn, c, kn)
                structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                dataFile = '../Processed_Data/DEEM_%s/PCA/%s/KPCAFiles.dat' % (sn, c)
                idxsFile = '../Processed_Data/DEEM_%s/PCA/%s/FPS-rKPCA.idxs' % (sn, c)
                trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
                validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
                dictKey = '%s-KPCA%s-%s-%s' % (sn, c, pn[0], kn[0])
                
                # Create directories
                subprocess.call(['mkdir', '-p', workDir])

                args = ['python', 'learningCurves.py',
                        '-structure', structureFile,
                        '-soap', dataFile,
                        '-idxs', idxsFile,
                        '-p', p,
                        '-Z', '14',
                        '-k', '5',
                        '-kernel', k,
                        '-width', str(hypers[dictKey][0]),
                        '-sigma', str(hypers[dictKey][1]),
                        '-zeta', '1',
                        '-ntrain'] + trainPts[sn] + \
                        ['-pcalearn'] + pcaPts[pn] + \
                        ['-train', trainFile,
                        '-validate', validateFile,
                        '-shuffle',
                        '-output', workDir]
                subprocess.call(args)

Current model: 1k 3.5 Energy Gaussian
Current model: 1k 3.5 Energy Linear
Current model: 1k 3.5 Volume Gaussian
Current model: 1k 3.5 Volume Linear
Current model: 1k 6.0 Energy Gaussian
Current model: 1k 6.0 Energy Linear
Current model: 1k 6.0 Volume Gaussian
Current model: 1k 6.0 Volume Linear
Current model: 10k 3.5 Energy Gaussian
Current model: 10k 3.5 Energy Linear
Current model: 10k 3.5 Volume Gaussian
Current model: 10k 3.5 Volume Linear
Current model: 10k 6.0 Energy Gaussian
Current model: 10k 6.0 Energy Linear
Current model: 10k 6.0 Volume Gaussian
Current model: 10k 6.0 Volume Linear


## Distances (done)

In [13]:
for ss, sn in zip(sampleSizes, sampleNames):
    for p, pn in zip(properties, propertyNames):
        for k, kn in zip(kernels, kernelNames):
            print 'Current model: %s %s %s' % (sn, pn, kn)
            workDir = '../Processed_Data/DEEM_%s/%s/Distances/%s' % (sn, pn, kn)
            structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
            dataFile = '../Processed_Data/DEEM_%s/Distances/distanceFiles.dat' % sn
            idxsFile = '../Processed_Data/DEEM_%s/Distances/FPS-rDistances.idxs' % sn
            trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
            validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
            dictKey = '%s-Dist-%s-%s' % (sn, pn[0], kn[0])
            
            # Create directories
            subprocess.call(['mkdir', '-p', workDir])

            args = ['python', 'learningCurves.py',
                    '-structure', structureFile,
                    '-soap', dataFile,
                    '-idxs', idxsFile,
                    '-p', p,
                    '-Z', '14',
                    '-k', '5',
                    '-kernel', k,
                    '-width', str(hypers[dictKey][0]),
                    '-sigma', str(hypers[dictKey][1]),
                    '-zeta', '1',
                    '-ntrain'] + trainPts[sn] + \
                    ['-train', trainFile,
                    '-validate', validateFile,
                    '-shuffle',
                    '-output', workDir]             
            subprocess.call(args)

Current model: 1k Energy Gaussian
Current model: 1k Energy Linear
Current model: 1k Volume Gaussian
Current model: 1k Volume Linear
Current model: 10k Energy Gaussian
Current model: 10k Energy Linear
Current model: 10k Volume Gaussian
Current model: 10k Volume Linear


## Angles (done)

In [15]:
for ss, sn in zip(sampleSizes, sampleNames):
    for p, pn in zip(properties, propertyNames):
        for k, kn in zip(kernels, kernelNames):
            print 'Current model: %s %s %s' % (sn, pn, kn)
            workDir = '../Processed_Data/DEEM_%s/%s/Angles/%s' % (sn, pn, kn)
            structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
            dataFile = '../Processed_Data/DEEM_%s/Angles/angleFiles.dat' % sn
            idxsFile = '../Processed_Data/DEEM_%s/Angles/FPS-rAngles.idxs' % sn
            trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
            validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
            dictKey = '%s-Ang-%s-%s' % (sn, pn[0], kn[0])
            
            # Create directories
            subprocess.call(['mkdir', '-p', workDir])

            args = ['python', 'learningCurves.py',
                    '-structure', structureFile,
                    '-soap', dataFile,
                    '-idxs', idxsFile,
                    '-p', p,
                    '-Z', '14',
                    '-k', '5',
                    '-kernel', k,
                    '-width', str(hypers[dictKey][0]),
                    '-sigma', str(hypers[dictKey][1]),
                    '-zeta', '1',
                    '-ntrain'] + trainPts[sn] + \
                    ['-train', trainFile,
                    '-validate', validateFile,
                    '-shuffle',
                    '-output', workDir]
            subprocess.call(args)

Current model: 1k Energy Gaussian
Current model: 1k Energy Linear
Current model: 1k Volume Gaussian
Current model: 1k Volume Linear
Current model: 10k Energy Gaussian
Current model: 10k Energy Linear
Current model: 10k Volume Gaussian
Current model: 10k Volume Linear


## Rings (done)

In [17]:
for ss, sn in zip(sampleSizes, sampleNames):
    for p, pn in zip(properties, propertyNames):
        for ks in ['King', 'Short']:
            for bd in ['Binary', 'Distribution']:
                for k, kn in zip(kernels, kernelNames):
                    print 'Current model: %s %s %s %s %s' % (sn, pn, ks, bd, kn)
                    workDir = '../Processed_Data/DEEM_%s/%s/Rings/%s/%s/%s' % (sn, pn, ks, bd, kn)
                    structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                    dataFile = '../Processed_Data/DEEM_%s/Rings/%s/%s/ringFiles.dat' % (sn, ks, bd)
                    idxsFile = '../Processed_Data/DEEM_%s/Rings/%s/%s/FPS-rRings.idxs' % (sn, ks, bd)
                    trainFile = '../Processed_Data/DEEM_%s/kTrain.idxs' % sn
                    validateFile = '../Processed_Data/DEEM_%s/kValidate.idxs' % sn
                    dictKey = '%s-%s%s-%s-%s' % (sn, ks, bd[0], pn[0], kn[0])
                    
                    # Create directories
                    subprocess.call(['mkdir', '-p', workDir])

                    args = ['python', 'learningCurves.py',
                            '-structure', structureFile,
                            '-soap', dataFile,
                            '-idxs', idxsFile,
                            '-p', p,
                            '-Z', '14',
                            '-k', '5',
                            '-kernel', k,
                            '-width', str(hypers[dictKey][0]),
                            '-sigma', str(hypers[dictKey][1]),
                            '-zeta', '1',
                            '-ntrain'] + trainPts[sn] + \
                            ['-train', trainFile,
                            '-validate', validateFile,
                            '-shuffle',
                            '-output', workDir]
                    subprocess.call(args)

Current model: 1k Energy King Binary Gaussian
Current model: 1k Energy King Binary Linear
Current model: 1k Energy King Distribution Gaussian
Current model: 1k Energy King Distribution Linear
Current model: 1k Energy Short Binary Gaussian
Current model: 1k Energy Short Binary Linear
Current model: 1k Energy Short Distribution Gaussian
Current model: 1k Energy Short Distribution Linear
Current model: 1k Volume King Binary Gaussian
Current model: 1k Volume King Binary Linear
Current model: 1k Volume King Distribution Gaussian
Current model: 1k Volume King Distribution Linear
Current model: 1k Volume Short Binary Gaussian
Current model: 1k Volume Short Binary Linear
Current model: 1k Volume Short Distribution Gaussian
Current model: 1k Volume Short Distribution Linear
Current model: 10k Energy King Binary Gaussian
Current model: 10k Energy King Binary Linear
Current model: 10k Energy King Distribution Gaussian
Current model: 10k Energy King Distribution Linear
Current model: 10k Energy Sh

In [ ]:
# Concatenate the KPCA for the Atlas
f = open('../Processed_Data/DEEM_10k/PCA/6.0/kpca3.dat', 'w')
a = subprocess.Popen(['python', 'npyConvert.py',
                 '-soap ../Processed_Data/DEEM_10k/PCA/6.0/KPCAFiles.dat',
                 '-convert', 'stdout'], stdout=subprocess.PIPE)
b = subprocess.Popen(['awk', "{print $1, $2, $3}"], stdin=a.stdout, stdout=f)

# Wait for a long time to make sure the file gets filled before we close it
time.sleep(300)

f.close()

In [ ]:
# Concatenate environment properties for the Atlas
# Figure out how many batches we have
n_batches = 0
with open('../Processed_Data/DEEM_10k/PCA/6.0/KPCAFiles.dat', 'r') as f:
    for line in f:
        n_batches += 1

energies = []
volumes = []
for i in range(0, n_batches):
    energies.append(np.loadtxt('../Processed_Data/DEEM_10k/Energy/6.0/Gaussian/envProperties-%d.dat' % i))
    volumes.append(np.loadtxt('../Processed_Data/DEEM_10k/Volume/6.0/Gaussian/envProperties-%d.dat' % i))

# Save the concatenated environment properties
np.savetxt('../Processed_Data/DEEM_10k/Energy/6.0/Gaussian/envProperties.dat', np.concatentate(energies))
np.savetxt('../Processed_Data/DEEM_10k/Volume/6.0/Gaussian/envProperties.dat', np.concatenate(volumes))

# Property Decomposition

## SOAP (done)

In [13]:
for ss, sn in zip(sampleSizes, sampleNames):
    for c in cutoffs:
        for p, pn in zip(properties, propertyNames):
            for k, kn in zip(kernels, kernelNames):
                print 'Current model: %s %s %s %s' % (sn, c, pn, kn)
                workDir = '../Processed_Data/DEEM_%s/%s/%s/%s' % (sn, pn, c, kn)
                structureFile = '../Raw_Data/DEEM_%s.xyz' % ss
                dataFile = '../Processed_Data/DEEM_%s/PCA/%s/SOAPFiles.dat' % (sn, c)
                idxsFile = '../Processed_Data/DEEM_%s/PCA/%s/FPS-rSOAP.idxs' % (sn, c)
                dictKey = '%s-SOAP%s-%s-%s' % (sn, c, pn[0], kn[0])
                
                # Create directories
                subprocess.call(['mkdir', '-p', workDir])

                subprocess.call(['python', 'propertyRegression.py',
                                '-structure', structureFile,
                                '-soap', dataFile,
                                '-idxs', idxsFile,
                                '-p', p, 
                                '-Z', '14',
                                '-k', k,
                                '-width', str(hypers[dictKey][0]),
                                '-sigma', str(hypers[dictKey][1]),
                                '-zeta', '1',
                                '-ntrain', ss,
                                '-env',
                                '-lowmem',
                                '-output', workDir])

Current model: 1k 3.5 Energy Gaussian
Current model: 1k 3.5 Energy Linear
Current model: 1k 3.5 Volume Gaussian
Current model: 1k 3.5 Volume Linear
Current model: 1k 6.0 Energy Gaussian
Current model: 1k 6.0 Energy Linear
Current model: 1k 6.0 Volume Gaussian
Current model: 1k 6.0 Volume Linear
Current model: 10k 3.5 Energy Gaussian
Current model: 10k 3.5 Energy Linear
Current model: 10k 3.5 Volume Gaussian
Current model: 10k 3.5 Volume Linear
Current model: 10k 6.0 Energy Gaussian
Current model: 10k 6.0 Energy Linear
Current model: 10k 6.0 Volume Gaussian
Current model: 10k 6.0 Volume Linear


# Selecting Scatter Plot Environments

## 1k sample

In [31]:
# Extract SOAP environment atoms (done)
subprocess.call(['python', 'atomLabels.py',
                '-input', '../Raw_Data/DEEM_1000.xyz',
                '-output', '../Processed_Data/DEEM_1k/atoms.dat',
                '-Z', '14',
                '-sp', 'Energy_per_Si', 'Filename'])

0

In [49]:
volP = np.loadtxt('../Processed_Data/DEEM_1k/Volume/6.0/Gaussian/envProperties-0.dat')
eP = np.loadtxt('../Processed_Data/DEEM_1k/Energy/6.0/Gaussian/envProperties-0.dat')

# Read DEEM xyz file
structure = aseIO.read('../Raw_Data/DEEM_1000.xyz', index=':')

# Replication tuple
r = (3, 3, 3)

# Sort properties to find "interesting" examples
volIdxs = np.argsort(volP)
eIdxs = np.argsort(eP)

low = 0
med = int(0.5*volP.size) # Volume and energy sizes are equal
high = -1

# Pick highest, median, and lowest property value environments
# and find the corresponding Si atoms
for x, xx, label in zip([volIdxs, eIdxs], [volP, eP], ['volume', 'energy']):
    for v, vv in zip([low, med, high], ['Low', 'Median', 'High']):
        
        # Read the atoms data
        h = subprocess.Popen(['head', '-%d' % (x[v]+1), '../Processed_Data/DEEM_1k/atoms.dat'], 
                              stdout=subprocess.PIPE)
        t = subprocess.Popen(['tail', '-1'], stdin=h.stdout, stdout=subprocess.PIPE)
        s = (t.communicate()[0]).strip().split()
        
        # Get the structure with the "interesting" environment
        subStructure = structure[int(s[7])]
        
        # Replicate the unit cell and find the atom index
        # in the middle of the replicated structure
        subStructueR = subStructure.repeat(r)
        newAtom = r[0]*r[1]*(r[2]/2) + r[0]*(r[1]/2) + r[0]/2
        newAtom *= len(subStructure)
        newAtom += int(s[1])
        
        print "===== %s %s =====" % (vv, label)
        print "Env. index: ", x[v]
        print "Env. %s: " % label, xx[x[v]]
        print "Structure: ", s[-1]
        print "Atom: ", s[1]
        print "Atom in 3x3x3: ", newAtom
        print ""

===== Low volume =====
Env. index:  15118
Env. volume:  31.192225804586315
Structure:  8147958.cif
Atom:  27
Atom in 3x3x3:  573

===== Median volume =====
Env. index:  25610
Env. volume:  51.15067096662824
Structure:  8216475.cif
Atom:  14
Atom in 3x3x3:  638

===== High volume =====
Env. index:  52051
Env. volume:  83.41127390215092
Structure:  8329015.cif
Atom:  117
Atom in 3x3x3:  741

===== Low energy =====
Env. index:  52156
Env. energy:  -54.702937507870956
Structure:  8329346.cif
Atom:  102
Atom in 3x3x3:  726

===== Median energy =====
Env. index:  32114
Env. energy:  -2.209387672424782
Structure:  8256857.cif
Atom:  70
Atom in 3x3x3:  616

===== High energy =====
Env. index:  52122
Env. energy:  87.07721357131959
Structure:  8329346.cif
Atom:  68
Atom in 3x3x3:  692



## 10k sample

In [ ]:
# Create directories
subprocess.call(['mkdir', '-p', '../Processed_Data/DEEM_10k/Environments'])

In [32]:
# Extract SOAP environment atoms (done)
subprocess.call(['python', 'atomLabels.py',
                '-input', '../Raw_Data/DEEM_10000.xyz',
                '-output', '../Processed_Data/DEEM_10k/atoms.dat',
                '-Z', '14',
                '-sp', 'Energy_per_Si', 'Filename'])

0

In [53]:
volP = []
eP = []

# Read DEEM xyz file
structure = aseIO.read('../Raw_Data/DEEM_10000.xyz', index=':')

# Replication tuple
r = (3, 3, 3)

# Read property data
# We have 10 batches
nBatches = 10
for i in range(0, nBatches):
    volPi = np.loadtxt('../Processed_Data/DEEM_10k/Volume/6.0/Gaussian/envProperties-%d.dat' % i)
    ePi = np.loadtxt('../Processed_Data/DEEM_10k/Energy/6.0/Gaussian/envProperties-%d.dat' % i)
    volP.append(volPi)
    eP.append(ePi)

volP = np.concatenate(volP)
eP = np.concatenate(eP)

# Sort properties to find "interesting" examples
volIdxs = np.argsort(volP)
eIdxs = np.argsort(eP)

low = 0
med = int(0.5*volP.size) # Volume and energy sizes are equal
high = -1

# Pick highest, median, and lowest property value environments
# and find the corresponding Si atoms
for x, xx, label in zip([volIdxs, eIdxs], [volP, eP], ['volume', 'energy']):
    for v, vv in zip([low, med, high], ['Low', 'Median', 'High']):
        
        # Read the atoms data
        h = subprocess.Popen(['head', '-%d' % (x[v]+1), '../Processed_Data/DEEM_10k/atoms.dat'], 
                              stdout=subprocess.PIPE)
        t = subprocess.Popen(['tail', '-1'], stdin=h.stdout, stdout=subprocess.PIPE)
        s = (t.communicate()[0]).strip().split()
        
        # Get the structure with the "interesting" environment
        subStructure = structure[int(s[7])]
        
        # Replicate the unit cell and find the atom index
        # in the middle of the replicated structure
        subStructureR = subStructure.repeat(r)
        newAtom = r[0]*r[1]*(r[2]/2) + r[0]*(r[1]/2) + r[0]/2
        newAtom *= len(subStructure)
        newAtom += int(s[1])
        
        # Save the replicated structure for further analysis
        aseIO.write('../Processed_Data/DEEM_10k/Environments/%s_%dx%dx%d.xyz' % (os.path.splitext(s[-1])[0],
                    r[0], r[1], r[2]), subStructureR, format='extxyz')
        
        print "===== %s %s =====" % (vv, label)
        print "Env. index: ", x[v]
        print "Env. %s: " % label, xx[x[v]]
        print "Structure: ", s[-1]
        print "Atom: ", s[1]
        print "Atom in 3x3x3: ", newAtom
        print ""

===== Low volume =====
Env. index:  416603
Env. volume:  23.78370619290581
Structure:  8285921.cif
Atom:  8
Atom in 3x3x3:  2192

===== Median volume =====
Env. index:  374965
Env. volume:  50.84192005263321
Structure:  8272545.cif
Atom:  58
Atom in 3x3x3:  4114

===== High volume =====
Env. index:  464212
Env. volume:  96.79278244932357
Structure:  8302881.cif
Atom:  33
Atom in 3x3x3:  2217

===== Low energy =====
Env. index:  491415
Env. energy:  -34.36726024466043
Structure:  8315425.cif
Atom:  32
Atom in 3x3x3:  2060

===== Median energy =====
Env. index:  236960
Env. energy:  -1.7357425685913768
Structure:  8199745.cif
Atom:  45
Atom in 3x3x3:  2229

===== High energy =====
Env. index:  380545
Env. energy:  386.01503800861246
Structure:  8274753.cif
Atom:  32
Atom in 3x3x3:  2294

